In [1]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from os.path import join
import pandas as pd
import cv2
import random

In [2]:
train_df = pd.read_csv('CKpaths.csv')
m = len(train_df)
train_df.head()

,neutral,peak,label,lm_neutral,lm_peak
0,../CK+/cohn-kanade-images/S063/002/S063_002_00...,../CK+/cohn-kanade-images/S063/002/S063_002_00...,../CK+/Emotion/S063/002/S063_002_00000023_emot...,../CK+/Landmarks/S063/002/S063_002_00000001_la...,../CK+/Landmarks/S063/002/S063_002_00000023_la...
1,../CK+/cohn-kanade-images/S063/001/S063_001_00...,../CK+/cohn-kanade-images/S063/001/S063_001_00...,../CK+/Emotion/S063/001/S063_001_00000013_emot...,../CK+/Landmarks/S063/001/S063_001_00000001_la...,../CK+/Landmarks/S063/001/S063_001_00000013_la...
2,../CK+/cohn-kanade-images/S071/004/S071_004_00...,../CK+/cohn-kanade-images/S071/004/S071_004_00...,../CK+/Emotion/S071/004/S071_004_00000028_emot...,../CK+/Landmarks/S071/004/S071_004_00000001_la...,../CK+/Landmarks/S071/004/S071_004_00000028_la...
3,../CK+/cohn-kanade-images/S071/002/S071_002_00...,../CK+/cohn-kanade-images/S071/002/S071_002_00...,../CK+/Emotion/S071/002/S071_002_00000020_emot...,../CK+/Landmarks/S071/002/S071_002_00000001_la...,../CK+/Landmarks/S071/002/S071_002_00000020_la...
4,../CK+/cohn-kanade-images/S071/006/S071_006_00...,../CK+/cohn-kanade-images/S071/006/S071_006_00...,../CK+/Emotion/S071/006/S071_006_00000014_emot...,../CK+/Landmarks/S071/006/S071_006_00000001_la...,../CK+/Landmarks/S071/006/S071_006_00000014_la...


In [3]:
def labelFromFile(labelpath):
    with open(labelpath, 'r') as file:
        return (int(float(file.read().rstrip().split()[0])))
train_df.label = list(map(labelFromFile, train_df.label))
train_df.to_csv('CKLabels_all.csv')

In [4]:
train_df.head()

,neutral,peak,label,lm_neutral,lm_peak
0,../CK+/cohn-kanade-images/S063/002/S063_002_00...,../CK+/cohn-kanade-images/S063/002/S063_002_00...,5,../CK+/Landmarks/S063/002/S063_002_00000001_la...,../CK+/Landmarks/S063/002/S063_002_00000023_la...
1,../CK+/cohn-kanade-images/S063/001/S063_001_00...,../CK+/cohn-kanade-images/S063/001/S063_001_00...,7,../CK+/Landmarks/S063/001/S063_001_00000001_la...,../CK+/Landmarks/S063/001/S063_001_00000013_la...
2,../CK+/cohn-kanade-images/S071/004/S071_004_00...,../CK+/cohn-kanade-images/S071/004/S071_004_00...,1,../CK+/Landmarks/S071/004/S071_004_00000001_la...,../CK+/Landmarks/S071/004/S071_004_00000028_la...
3,../CK+/cohn-kanade-images/S071/002/S071_002_00...,../CK+/cohn-kanade-images/S071/002/S071_002_00...,6,../CK+/Landmarks/S071/002/S071_002_00000001_la...,../CK+/Landmarks/S071/002/S071_002_00000020_la...
4,../CK+/cohn-kanade-images/S071/006/S071_006_00...,../CK+/cohn-kanade-images/S071/006/S071_006_00...,3,../CK+/Landmarks/S071/006/S071_006_00000001_la...,../CK+/Landmarks/S071/006/S071_006_00000014_la...


In [4]:
def landmarksFromFile(lmpath):
    with open(lmpath, 'r') as file:
        # read and store landmarks
        L = file.read().split()
        L = list(map(float, L))
        
        # separate X and Y
        X, Y = np.empty(68), np.empty(68)
        for i, num in enumerate(L):
            if i%2==0:
                X[i//2] = num
            else:
                Y[i//2] = num
        return X, Y
    
def mean_normalized(X): # returns z-score for each value
    mux = np.mean(X, axis=1).reshape(-1,1)
    sigmax = np.std(X, axis=1).reshape(-1,1)
    return (X-mux)/sigmax

def l2_normalized(X):
    norms = np.linalg.norm(X, axis=1).reshape(-1,1)
    return X/norms

def pairwiseDistances(X):
    m, n = tuple(X.shape)
    n_dash = n*(n-1)//2
    R = np.empty((m,n_dash))
    index = 0
    for i in range(n):
        for j in range(i+1, n): # pair ij
            R[:, [index]] = X[:, [i]] - X[:,[j]]
            index += 1
    print(n_dash==index)
    return R
                 
def allLandmarks(lmpathlist): # returns shift invariant landmarks
    X, Y = np.empty((m, 68)), np.empty((m, 68))
    for i, lmpath in enumerate(lmpathlist):
        X[i], Y[i] = landmarksFromFile(lmpath)
        
    # get pair-wise distances
    return pairwiseDistances(X), pairwiseDistances(Y)

In [5]:
# get pairwise distances from neutral
NX, NY = allLandmarks(train_df.lm_neutral)
PX, PY = allLandmarks(train_df.lm_peak)
# store the landmarks as data
P = np.sqrt(PX**2 + PY**2)
np.save("peak", P)
DX, DY = PX-NX, PY-NY
F = np.hstack((DX,DY))

In [6]:
m,n = tuple(F.shape)
# shuffle rows
r = np.random.permutation(m)
F = F[r]
# save the features in a file
np.save('features', F)

In [7]:
T = np.array(train_df.label)
np.save('targets', T)

In [8]:
print(P.shape)

(326, 2278)
